# scan different paramters for the first low res registartion

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from tifffile import imwrite as tif_imwrite
from tifffile import imread as tif_imread
from IPython.display import display, HTML
import numpy as np
from tqdm.auto import tqdm
import os
import hjson
from pathlib import Path
import os
sys.path.append("../../")
from registrations_funcs import HCR_confocal_imaging, register_lowres, verify_rounds

display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
import socket

# Path for bigstream unless you did pip install
sys.path = [fr"\\nasquatch\data\2p\jonna\Code_Python\Notebooks_Jonna\BigStream\bigstream_github"] + sys.path 
sys.path = [fr"C:\Users\jonna\Notebooks_Jonna\BigStream\bigstream_github"] + sys.path 
sys.path = [fr'{os.getcwd()}/bigstream_github'] + sys.path
sys.path = ["/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_github"] + sys.path 



hjson_file = fr'/home/oamsalem/code/Projects/EZfish/ezfish_pipeline/examples/CIM132.hjson'
manifest = hjson.load(open(hjson_file, 'r'))

output_folder = Path(manifest['base_path']) / manifest['mouse_name'] / 'OUTPUT' / 'HCR' / 'registrations' 


round_to_rounds, reference_round = verify_rounds(manifest)
reference_round['round']

All files found
Rounds available for register:
02 ['DAPI', 'FOXP2', 'CCK', 'SST']
03 ['DAPI', 'CALCA', 'TAC1']
04 ['DAPI', 'GRP', 'SYT10', 'CHAT']


'01'

### PARAMETERS

In [2]:
# Choose the round to register
HCR_round_to_register = '02'# 02,03...?

assert HCR_round_to_register in round_to_rounds.keys(), f"round {HCR_round_to_register} not found in rounds to register"



### Image parameters

In [3]:
mov_round = round_to_rounds[HCR_round_to_register]
# File names
HCR_fix_image_path = reference_round['image_path'] # The fix image that all other rounds will be registerd to (include all channels!)
HCR_mov_image_path = mov_round['image_path'] # The image that will be registered to the fix image (include all channels!)

round_folder_name = f"HCR{HCR_round_to_register}_to_HCR{reference_round['round']}"

# resolution of the images
fix_image_spacing = np.array(reference_round['resolution']) # Y,X,Z
mov_image_spacing = np.array(mov_round['resolution']) # Y,X,Z

# spatial downsampling, probably no need to test. (Changed x and y from 3 to 2 for CIM round 5)
red_mut_x = manifest['HCR_to_HCR_params']['red_mut_x']
red_mut_y = manifest['HCR_to_HCR_params']['red_mut_y']
red_mut_z = manifest['HCR_to_HCR_params']['red_mut_z']



### bigstream global parameters

In [4]:
# radius for local ransac for each spot (in pixles for LOW res image) - how many pixels to look around a spot for match
g_radii_z = [1,5,15] #Typical values: 1,3,5
g_radii_xy = [5, 10, 25, 35] #Typical values: 5, 8, 10

# when searching for blobs for ransac, what is the radius for the search alogorithem (in pixles for low res image)
g_bsize_s_b = [[3,10], [3,8], [5,15], [5, 20], [2,30],[4,30]]

# what is the minumum correlation for a spot to be considered a match
g_m_tresholds = [0.1, 0.2, 0.3, 0.4] 


#### Load the images

In [5]:
# Take first channel, I always assume that the first channel is what we want to register
fix_highres = tif_imread(HCR_fix_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z
mov_highres = tif_imread(HCR_mov_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z


fix_lowres = fix_highres[::red_mut_y,::red_mut_x,::red_mut_z]
fix_lowres_spacing = fix_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])


mov_lowres = mov_highres[::red_mut_y,::red_mut_x,::red_mut_z]
mov_lowres_spacing = mov_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])

# Print the dimensions of the high-resolution and low-resolution images
print(f"The dimensions of the high-resolution fixed image are: {fix_highres.shape}")
print(f"The dimensions of the high-resolution moving image are: {mov_highres.shape}")
print(f"The dimensions of the low-resolution moving image are: {mov_lowres.shape}")

The dimensions of the high-resolution fixed image are: (2922, 2914, 71)
The dimensions of the high-resolution moving image are: (3437, 3555, 77)
The dimensions of the low-resolution moving image are: (1719, 1778, 39)


In [11]:
success_n = 0
write_dir = output_folder / output_folder_name / 'global_parms_scan' 
write_dir.mkdir(exist_ok=True,parents=True)
write_directory = str(write_dir)
print(write_directory)
# saving 1 version of the mov lowres
tif_imwrite(f'{write_directory}/mov_lowres.tiff', mov_lowres.transpose(2,1,0))
njobs = len(g_radii_z)*len(g_radii_xy)*len(g_bsize_s_b)*len(g_m_tresholds)
pbar = tqdm(total=njobs)
for g_radius_z in g_radii_z:
  for g_radius_xy in g_radii_xy:
    for g_bsize_s, g_bsize_b in g_bsize_s_b:
      for g_m_treshold in g_m_tresholds:
        
        fname = f"rz{g_radius_z}_rxy{g_radius_xy}_bs{g_bsize_s}to{g_bsize_b}_mt{g_m_treshold}"

        global_ransac_kwargs = {'cc_radius':(g_radius_xy,g_radius_xy,g_radius_z),
                                'blob_sizes':[g_bsize_s, g_bsize_b],
                                'match_threshold':g_m_treshold}

        output = register_lowres(
            fix_lowres, mov_lowres,
            fix_lowres_spacing,
            mov_lowres_spacing,
            global_ransac_kwargs=global_ransac_kwargs,
            write_directory=write_directory,
            fname=fname
        )
        
        pbar.update(1)
        pbar.set_description(f"Success: {success_n}/{pbar.n}")
        if output is not None:
            success_n+=1
print(f' got {success_n} registration to work!')

/mnt/bananas/data/ezfish/jonna/RG026/OUTPUT/HCR/registrations/global_parms_scan


  0%|          | 0/288 [00:00<?, ?it/s]

1, blob_min=12 , blob_max=48
Fri Aug 16 11:01:32 2024 computing fixed spots
Fri Aug 16 11:01:32 2024 fixed spots detection using {'num_sigma': 7, 'exclude_border': (5, 5, 1)}
Fri Aug 16 11:01:32 2024 Start spot detection ((3, 3, 3), (10, 10, 10)) {'exclude_border': (5, 5, 1), 'num_sigma': 7, 'min_sigma': array([1.73205081, 1.73205081, 1.73205081]), 'max_sigma': array([5.77350269, 5.77350269, 5.77350269]), 'threshold': None, 'threshold_rel': 0.1}
Fri Aug 16 11:01:40 2024 Spot detection ((3, 3, 3), (10, 10, 10)) completed in 8.219963312149048s
Fri Aug 16 11:01:40 2024 found 4386 fixed spots
Fri Aug 16 11:01:40 2024 computing moving spots
Fri Aug 16 11:01:40 2024 moving spots detection using {'num_sigma': 7, 'exclude_border': (5, 5, 1)}
Fri Aug 16 11:01:40 2024 Start spot detection ((3, 3, 3), (10, 10, 10)) {'exclude_border': (5, 5, 1), 'num_sigma': 7, 'min_sigma': array([1.73205081, 1.73205081, 1.73205081]), 'max_sigma': array([5.77350269, 5.77350269, 5.77350269]), 'threshold': None, 'th

In [15]:
from bigstream.align import feature_point_ransac_affine_align
from bigstream.application_pipelines import easifish_registration_pipeline
from bigstream.transform import apply_transform

[autoreload of registrations_funcs failed: Traceback (most recent call last):
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/oamsalem/code/Projects/EZfish/ezfish_pipeline/src/processing_notebooks/HCR_rounds/../../registrations_funcs.py", 

ModuleNotFoundError: No module named 'bigstream.align'

In [1]:
import SimpleITK as sitk
import numpy as np
import os
from pathlib import Path
import sys
# Path for bigstream unless you did pip install
sys.path = ["/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_github"] + sys.path 

from bigstream.align import feature_point_ransac_affine_align
from bigstream.application_pipelines import easifish_registration_pipeline
from bigstream.transform import apply_transform

In [2]:
ls /mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_github

bigstream/       Dockerfile*  notebooks/  resources/  setup.py*
conda-env.yaml*  LICENSE.md*  README.md*  scripts/


In [7]:
from tqdm.auto import tqdm
/

()

In [9]:
from registrations_funcs import HCR_confocal_imaging, register_lowres